# Train longer
 
* Train a CNN for 10000 epochs with LR of 1e-5
* PC crashed at 6000 epochs, took around 1.5 days
* Results are on [wandb](https://wandb.ai/amitagni/candlestick-CNN/runs/27gxv7mr?workspace=user-amitagni)


In [ ]:


modelBase = {}
modelBase['name'] = 'Base'
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['activation'] = 'relu'


modelBase['convLayerMultiplier'] = 1

modelBase['poolingLayer'] = 'MaxPooling2D'
modelBase['padding'] = 'same'

modelBase['denseLayers'] = 2
modelBase['units'] = 128
modelBase['activation'] = 'relu'

#with Dropout and BN
modelBase_with_Dropout = modelBase.copy()
modelBase_with_Dropout['name'] = 'modelBase_with_Dropout'
modelBase_with_Dropout['batchnormalization'] = False
modelBase_with_Dropout['dropout'] = 0.01

modelBase_with_Dropout['kernelSize'] = (3,3)
modelBase_with_Dropout['filters'] = [10,15,20,25,30,35,40,45,50,55,60]
modelBase_with_Dropout['poolSize'] = (6,6)

df = pd.DataFrame()

for imgs in [85000]:
    for lr in [1e-5]:

        start_time = time.time()

        set_x = set_x[:imgs]
        set_y = set_y[:imgs]
        
        model = createCNN(modelBase_with_Dropout)
        
        model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr)
                ,loss=tf.keras.losses.binary_crossentropy
                ,metrics=[METRICS])

        history = model.fit(set_x,set_y
                        ,batch_size = 128
                        ,epochs=10000
                        ,verbose=0   
                        ,validation_data=(test_set_x,test_set_y)
                        ,validation_freq = 50
                        ,callbacks=[WandbCallback()]                                        
                        )

        temp = pd.DataFrame(history.history).rename_axis("epoch")
        
        temp['elapsed'] = round((time.time() - start_time)/60,2)

        var_params = "Deeper_" + "imgs_" + str(imgs) + "_LR_"  + str(lr)

        temp['params'] = var_params

        print("Elapsed time " + str(round((time.time() - start_time)/60,2)) + var_params)
